# Download dataset in Google Colab

In [9]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# If we're in Colab, we need to download the data folder from GitHub
if IN_COLAB:
    import os
    
    # Check if the data directory already exists or not
    if not 'data' in os.listdir():
        print("Downloading data...")
        os.makedirs("data", exist_ok=True)

        for file in ["Background.csv", "EE.csv", "Throughput.csv"]:
            os.system(f"wget https://github.com/italoseara/MOSAIC-Prototype/raw/main/data/{file} -O data/{file}")
            print(f"Downloaded {file}")

DATA_DIR = "./data" if IN_COLAB else "../data"

# Importing Required Libraries

In [10]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from typing import Literal

# Constants

In [11]:
ZERO_POINT = -48.6 # erg/s/cm^2/Hz
PLANCK_CONSTANT = 6.63e-34 # J.s
VELOCITY_OF_LIGHT = 299792458 # m/s

# Parameters

In [12]:
# Instrument
class MOS_VIS:
    RESOLUTION: int = 5000
    APERTURE_DIAMETER: float = 0.7 # arcsec
    FIBERS_PER_APERTURE: 7
    SPACIAL_SAMPLING: int = 5 # pixels
    SPECTRAL_SAMPLING: int = 5 # pixels
    PIXELS_PER_OBJECT: int = 175

class MOS_NIR:
    RESOLUTION: int = 5000
    APERTURE_DIAMETER: float = 0.6 # arcsec
    FIBERS_PER_APERTURE: 7
    SPACIAL_SAMPLING: int = 3 # pixels
    SPECTRAL_SAMPLING: int = 3 # pixels
    PIXELS_PER_OBJECT: int = 63

class IFU:
    RESOLUTION: int = 5000
    SPAXEL_SIZE: float = 0.15 # arcsec
    SPACIAL_SAMPLING: int = 3 # pixels
    SPECTRAL_SAMPLING: int = 3 # pixels
    SPAXELS_PER_SPAXEL: int = 27

MOS = { 'VIS': MOS_VIS, 'NIR': MOS_NIR }

# Detector
class DETECTOR_VIS:
    DARK_CURRENT: float = 10/3600 # e/s/pixel
    RON = 3 # e/pixel

class DETECTOR_NIR:
    DARK_CURRENT: float = 0.01 # e/s/pixel
    RON = 3 # e/pixel

DETECTORS = { 'VIS': DETECTOR_VIS, 'NIR': DETECTOR_NIR }

# Telescope
ELT_DIAMETER: float = 38.542 # m

# Initialize Tables

In [13]:
class LookupTable:
    """A class to represent a lookup table for retrieving data from a CSV file."""

    _df: pd.DataFrame
    _col: str
    
    def __init__(self, csv_file: str, col: str, **kwargs) -> None:
        self._df = pd.read_csv(csv_file, **kwargs)
        self._col = col

    def get(self, key: float) -> dict[str, float]:
        closest_index = self._df[self._col].sub(key).abs().idxmin()
        return self._df.loc[closest_index].to_dict()

gt_table = LookupTable(f"{DATA_DIR}/Throughput.csv", "Wavelength LR")
ee_table = LookupTable(f"{DATA_DIR}/EE.csv", "Wavelength")
bg_table = LookupTable(f"{DATA_DIR}/Background.csv", "Wavelength")

# Signal to noise ratio

Let:
- $c$ be the speed of light in $m/s$.
- $h$ be the Planck constant in $J.s$.
- $t$ be the detector integration time in $s$.
- $n$ be the number of DITs.
- $\lambda$ be the wavelength in $\mu m$.
- $m$ be the magnitude of the source in $AB mag$.
- $m_0$ be the zero point magnitude in $AB mag$.
- $T_g$ be the global throughput in %.
- $E_e$ be the enclosed energy in %.
- $N_{out}$ be the readout noise in $e$.
- $N_{sky}$ be the sky background noise in $e/s/pixel$.
- $r_{elt}$ be the radius of the ELT in $m$.
- $r_{ap}$ be the aperture radius of the instrument in $arcsec$.
- $R$ be the resolution of the instrument.
- $E_{ph}$ be the energy of a photon in $J$.
- $F$ be the flux in $ph/s^2$
- $C$ be the number of counts in $ph/s$.
- $F_{bg}$ be the background flux in $ph/s^2$.
- $C_{bg}$ be the number of background counts in $ph/s$.
- $I_{dark}$ be the dark current in $e/s/pixel$.
- $P_{obj}$ be the pixels per object.
- $S/N$ be the signal to noise ratio.

$$E_{ph} = \frac{c \cdot h}{10 \cdot \lambda}$$

&nbsp;

$$F = \frac{10^{-0.4 (m - m_0)}}{E_{ph} \cdot \lambda \cdot R} \cdot 10^{-4} \cdot c \cdot \pi r_{elt}^2 \cdot E_e \cdot T_g$$

$$C = F \cdot t$$

&nbsp;

$$F_{bg} = \lambda \cdot N_{sky} \cdot \pi r_{elt}^2 \cdot \pi r_{ap}^2 \cdot T_g \cdot R^{-1}$$

$$C_{bg} = F_{bg} \cdot t$$

&nbsp;

$$S/N = \frac{n \sqrt{C}}{C + C_{bg} + N_{out}^2 \cdot P_{obj} + I_{dark} \cdot P_{obj} \cdot t}$$

In [14]:
def signal_to_noise(
    dit: float | np.ndarray,
    ndit: int,
    source_counts: float,
    sky_counts: float,
    dark_current: float,
    ron: float,
    npix: int,
) -> float | np.ndarray:
    """Calculate the signal-to-noise ratio for a given observation."""

    signal = source_counts * np.sqrt(ndit)
    noise = np.sqrt(source_counts + sky_counts + ron**2 * npix + dark_current * dit * npix)
    return signal / noise

def source_flux(
    magnitude: float,
    wavelength: float,
    enclosed_energy: float,
    global_throughput: float,
    resolution: int
) -> float:
    """Calculate the photon flux of the source."""

    energy_ph = (VELOCITY_OF_LIGHT * PLANCK_CONSTANT) / (wavelength * 10)
    area_elt = np.pi * (ELT_DIAMETER / 2)**2

    return (
        10**(-0.4 * (magnitude - ZERO_POINT)) / 
        (energy_ph * wavelength * resolution) * 
        10**-4 * VELOCITY_OF_LIGHT * area_elt * 
        enclosed_energy * global_throughput
    )


In [15]:
def interactive(
    ndit: int,
    dit: int,
    wavelength: float,
    source: float,
    sky_condition: Literal["Sky ALL", "Sky NO MOON", "Sky ALL+Th", "Sky NO MOON+Th"], 
    instrument: Literal["MOS VIS", "MOS NIR", "IFU"],
) -> float:
    """Calculate the signal-to-noise ratio for a given observation."""

    if instrument == "IFU":
        global_throughput = gt_table.get(wavelength)[f"NIR-LR"]
        enclosed_energy = ee_table.get(wavelength * 1000)[f"EE-IFU-ZA45 Durham"]
        sky_noise = bg_table.get(wavelength * 1000)[sky_condition]

        print("Global throughput: {:.2f}%".format(global_throughput * 100))
        print("Enclosed energy: {:.2f}%".format(enclosed_energy * 100))
        print("Sky noise: {:.2f} e/s/pixel".format(sky_noise))

        flux = source_flux(source, wavelength, enclosed_energy, global_throughput, IFU.RESOLUTION)
        counts = flux * dit

        print("Flux: {:.2f} ph/s/DIT".format(flux))
        print("Counts: {:.2f} ph/DIT".format(counts))

        area_elt = np.pi * (ELT_DIAMETER / 2)**2

        sky_flux = sky_noise * area_elt * (wavelength / IFU.RESOLUTION) * IFU.SPAXEL_SIZE**2 * global_throughput
        sky_counts = sky_flux * dit

        print(f"Background flux: {sky_flux:.2f} ph/s/DIT")
        print(f"Background counts: {sky_counts:.2f} ph/DIT")

        snr = signal_to_noise(
            dit,
            ndit,
            counts,
            sky_counts,
            DETECTORS["NIR"].DARK_CURRENT,
            DETECTORS["NIR"].RON,
            IFU.SPAXELS_PER_SPAXEL
        )

        print(f"\nS/R: {snr:.2f}")
        return

    band = instrument.split(" ")[1]

    # Get the throughput, enclosed energy and sky noise for the given wavelength
    global_throughput = gt_table.get(wavelength)[f"{band}-LR"]
    enclosed_energy = ee_table.get(wavelength * 1000)[f"EE-LR-ZA45 Durham {band}"]
    sky_noise = bg_table.get(wavelength * 1000)[sky_condition]

    print(f"Global throughput: {global_throughput * 100:.2f}%")
    print(f"Enclosed energy: {enclosed_energy * 100:.2f}%")
    print(f"Sky noise: {sky_noise:.2f} e/s/pixel")

    # Calculate the flux
    flux = source_flux(source, wavelength, enclosed_energy, global_throughput, MOS[band].RESOLUTION)
    counts = flux * dit

    print(f"Flux: {flux:.2f} ph/s/DIT")
    print(f"Counts: {counts:.2f} ph/DIT")
    
    # Calculate the background noise flux
    area_elt = np.pi * (ELT_DIAMETER / 2)**2
    area_ap = np.pi * (MOS[band].APERTURE_DIAMETER / 2)**2

    sky_flux = wavelength * sky_noise * area_elt * area_ap * global_throughput * (1/MOS[band].RESOLUTION)
    sky_counts = sky_flux * dit

    print(f"Background flux: {sky_flux:.2f} ph/s/DIT")
    print(f"Background counts: {sky_counts:.2f} ph/DIT")

    snr = signal_to_noise(
        dit,
        ndit,
        counts,
        sky_counts,
        DETECTORS[band].DARK_CURRENT,
        DETECTORS[band].RON,
        MOS[band].PIXELS_PER_OBJECT
    )
    
    print(f"\nS/R: {snr:.2f}")
    return snr

# Interactive User Inputs

In [16]:
style = {'description_width': '150px'}
layout = {'width': '400px'}

ndit_w = widgets.IntSlider(
    value=60, min=1, max=100, step=1,
    description="Number of exposures", 
    style=style, layout=layout)
dit_w = widgets.IntSlider(
    value=300, min=1, max=3600, step=1,
    description="Exposure time (s)", 
    style=style, layout=layout)
wavelength_w = widgets.FloatSlider(
    value=0.45, min=0.3, max=1.9, step=0.05,
    description="Wavelength (um)", 
    style=style, layout=layout)
source_w = widgets.FloatSlider(
    value=21, min=0, max=100, step=1, 
    description="Source magnitude",
    style=style, layout=layout)
sky_w = widgets.Dropdown(
    value="Sky NO MOON+Th",
    options=["Sky ALL", "Sky NO MOON", "Sky ALL+Th", "Sky NO MOON+Th"],
    description="Sky condition",
    style=style, layout=layout)
instrument_w = widgets.Dropdown(
    options=["MOS VIS", "MOS NIR", "IFU"],
    description="Instrument",
    style=style, layout=layout)

widgets.interactive(
    interactive,
    ndit=ndit_w, dit=dit_w,
    wavelength=wavelength_w,
    source=source_w,
    sky_condition=sky_w,
    instrument=instrument_w)

interactive(children=(IntSlider(value=60, description='Number of exposures', layout=Layout(width='400px'), min…